1. Monte um passo a passo para o algoritmo RF
2. Explique com suas palavras o Random forest
3. Qual a diferença entre Bagging e Random Forest?
4. (Opcional) Implementar em python o Random Forest– Bootstrap– Feature selection– Modelagem com Decision trees– Agregação

### 1. Monte um passo a passo para o algoritmo RF

1. Amostragem Bootstrap (Bagging)
> - Nesta etapa, o algoritmo cria várias amostras do conjunto de treino original com substituição (bootstrap)

2. Construção das Árvores de Decisão
> - Para cada amostra bootstrap, o algoritmo constrói uma árvore de decisão. No entanto, o diferencial do Random Forest em relação ao bagging é que, em cada nó de decisão da árvore, o algoritmo seleciona de forma aleatória as features. Isso cria árvores mais diversas, evitando a correlação entre elas.

3. Treinamento das Árvores
> - Cada árvore de decisão é treinada de forma independente utilizando a amostra bootstrap correspondente. O objetivo é criar um conjunto de árvores que realizam previsões a partir de diferentes partes do conjunto de dados, o que aumenta a robustez do modelo.

4. Previsão com o Conjunto de Árvores
> - Após o treinamento, o algoritmo usa todas as árvores para fazer previsões. Para problemas de classificação, cada árvore faz uma previsão, e o resultado final é obtido pela votação por maioria entre as árvores. Para problemas de regressão, a média das previsões das árvores é utilizada.

5. Avaliação do Desempenho
> - Avaliar a performance do Random Forest usando métricas apropriadas

### 2. Explique com suas palavras o Random forest

> - O RF é um algoritmo de machine learning baseado em Decicion Tree que utiliza a tecnica **bagging** melhorando a precisão e reduzindo o risco de overfitting. Ele cria múltiplas Decision Trees e, em vez de treinar cada árvore com o dataset completo, utiliza features aleatória de amostras e variáveis.


### 3. Qual a diferença entre Bagging e Random Forest?

> - A diferença principal esta no tratamento de features durante a construção das árvores

> - **RANDOM FOREST**:
>> - Durante a construção de cada nó da arvore, o RF seleciona um subconjunto aleatorio de variáei e vez de usar todas as que estão disponiveis, garantindo que a árores sejam menos correlacionadas.

> - **BAGGING**:
>> - Treina conjuntos independentes, que podem ser Decision Trees ou outros tipos de modelos. Em métodos de bagging simples como para Decision Trees, todas as features são consideradas para cada árvore.

### 4. Implementar em python o Random Forest– Bootstrap– Feature selection– Modelagem com Decision trees– Agregação

In [13]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [31]:
# Carregar o dataset de câncer de mama
data = load_breast_cancer()
X, y = data.data, data.target

# Dividir o dataset em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=987645231)


In [15]:
def bootstrap_sample(X, y):
    n_samples = X.shape[0]
    indices = np.random.choice(n_samples, size=n_samples, replace=True)
    return X[indices], y[indices]

def random_features(X, max_features):
    n_features = X.shape[1]
    feature_indices = np.random.choice(n_features, size=max_features, replace=False)
    return feature_indices

def train_random_forest(X_train, y_train, n_estimators=10, max_features=None):
    models = []
    for _ in range(n_estimators):
        # Gerar amostra bootstrap
        X_sample, y_sample = bootstrap_sample(X_train, y_train)
        
        # Escolher aleatoriamente um subconjunto de features
        if max_features is None:
            max_features = int(np.sqrt(X_train.shape[1]))  # Critério comum em Random Forest
        
        feature_indices = random_features(X_sample, max_features)
        
        # Treinar árvore de decisão com um subconjunto de features
        model = DecisionTreeClassifier()
        model.fit(X_sample[:, feature_indices], y_sample)  # Usando apenas as features selecionadas
        models.append((model, feature_indices))  # Armazenar o modelo e as features usadas
    
    return models

def random_forest_predict(models, X_test):
    predictions = []
    for model, feature_indices in models:
        # Fazer a previsão apenas com as features que o modelo foi treinado
        pred = model.predict(X_test[:, feature_indices])
        predictions.append(pred)
    
    # Votação por maioria entre as árvores
    predictions = np.array(predictions)
    majority_votes = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=predictions)
    return majority_votes


In [33]:
# Parâmetros do modelo
n_estimators = 300  # Número de árvores
max_features = int(np.sqrt(X_train.shape[1]))  # Número de features aleatórias por nó

# Treinar a Random Forest
models = train_random_forest(X_train, y_train, n_estimators, max_features)

# Fazer previsões com o modelo Random Forest
y_pred = random_forest_predict(models, X_test)

# Avaliar a acurácia
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo Random Forest: {accuracy * 100:.2f}%")


Acurácia do modelo Random Forest: 94.74%
